# Generating artificial data

### This script generates and saves synthetic time series data using AR, MA, and ARMA models with optional trend and seasonality. It creates 4000 time series per class, stores them in a structured directory, and saves them as images. The script also verifies the number of images per class.

In [10]:
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
import os


In [4]:
import numpy as np
import os

np.random.seed(1234)  # For reproducibility
n_series = 4000   # Number of time series per class
n_points = 500    # Number of data points in each time series



output_dir = os.path.expanduser("~/timeseries_data")  # Creates the directory in your home folder
os.makedirs(output_dir, exist_ok=True)


#### AR, MA and ARMA Model

In [6]:
# Function to generate AR, MA, or ARMA data with optional trend and seasonality
def generate_time_series(model_type, order, n_points, trend_strength=0.1, seasonality_amplitude=0.5, seasonality_period=50, include_trend=True):
   if model_type == 'AR':
       params = np.random.uniform(-0.5, 0.5, size=order)
       ar = np.r_[1, -params]
       ma = np.array([1])
   elif model_type == 'MA':
       params = np.random.uniform(-0.5, 0.5, size=order)
       ar = np.array([1])
       ma = np.r_[1, params]
   elif model_type == 'ARMA':
       ar_params = np.random.uniform(-0.5, 0.5, size=order)
       ma_params = np.random.uniform(-0.5, 0.5, size=order)
       ar = np.r_[1, -ar_params]
       ma = np.r_[1, ma_params]
   else:
       raise ValueError("Invalid model type. Use 'AR', 'MA', or 'ARMA'.")
   
   
      # Generate the process
   process = sm.tsa.ArmaProcess(ar, ma)
   data = process.generate_sample(nsample=n_points)
   if include_trend:
       trend = np.linspace(0, trend_strength * n_points, n_points)
       seasonality = seasonality_amplitude * np.sin(2 * np.pi * np.arange(n_points) / seasonality_period)
       data += trend + seasonality
   else:
       seasonality = seasonality_amplitude * np.sin(2 * np.pi * np.arange(n_points) / seasonality_period)
       data += seasonality
   return data

#### Models for orders 1-3 with and without trend 

In [15]:
# Loop to generate and save time series plots for each class
model_types = ['AR', 'MA', 'ARMA']
orders = [1, 2, 3]
for model_type in model_types:
   for order in orders:
       for include_trend in [True, False]:
           class_label = f'{model_type}_{order}_with_trend' if include_trend else f'{model_type}_{order}_without_trend'
           class_dir = os.path.join(output_dir, class_label)
           os.makedirs(class_dir, exist_ok=True)
           for i in range(n_series):
               data = generate_time_series(model_type, order, n_points, include_trend=include_trend)
               
               # Plotting the time series
               plt.figure(figsize=(8, 4))
               plt.plot(data)
               plt.axis('off')  # Turn off axes for a clean image
               plt.savefig(os.path.join(class_dir, f'Series_{i+1}.png'), bbox_inches='tight', pad_inches=0)
               plt.close()
print("Time series generation completed. Time series are saved in the 'time_series_data' directory.")

: 

#### Generating images for remaining 4 classes since the kernel crashed in the previous cell after 14 classes

In [7]:
# Adjusted classes for ARMA 2 and 3 with and without trend
model_type = 'ARMA'
orders = [2, 3]
for order in orders:
    for include_trend in [True, False]:
        class_label = f'{model_type}_{order}_with_trend' if include_trend else f'{model_type}_{order}_without_trend'
        class_dir = os.path.join(output_dir, class_label)
        os.makedirs(class_dir, exist_ok=True)
        for i in range(n_series):
            data = generate_time_series(model_type, order, n_points, include_trend=include_trend)
            # Plotting the time series
            plt.figure(figsize=(8, 4))
            plt.plot(data)
            plt.axis('off')  # Turn off axes for a clean image
            plt.savefig(os.path.join(class_dir, f'Series_{i+1}.png'), bbox_inches='tight', pad_inches=0)
            plt.close()

# Code to check the number of images in each folder
folder_status = {}
for folder_name in os.listdir(output_dir):
    folder_path = os.path.join(output_dir, folder_name)
    if os.path.isdir(folder_path):
        folder_status[folder_name] = len(os.listdir(folder_path))

# Display folder status
for class_label, image_count in folder_status.items():
    print(f"{class_label}: {image_count} images")

ARMA_1_without_trend: 4000 images
ARMA_2_with_trend: 4000 images
MA_3_with_trend: 4000 images
ARMA_2_without_trend: 4000 images
AR_3_without_trend: 4000 images
MA_1_without_trend: 4000 images
MA_1_with_trend: 4000 images
AR_2_with_trend: 4000 images
MA_2_without_trend: 4000 images
AR_1_with_trend: 4000 images
MA_2_with_trend: 4000 images
MA_3_without_trend: 4000 images
ARMA_3_with_trend: 4000 images
AR_3_with_trend: 4000 images
AR_2_without_trend: 4000 images
ARMA_3_without_trend: 4000 images
AR_1_without_trend: 4000 images
ARMA_1_with_trend: 4000 images


# Loading and preprocessing the data

### This script loads and processes time series images for classification using deep learning. It resizes images, normalizes pixel values, and maps class labels. A subset of classes is selected, and the dataset is split into training, validation, and test sets using stratified sampling. A custom data generator is implemented to efficiently handle indexed batches for training a neural network.

In [1]:
import os
import numpy as np
from PIL import Image
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import BatchNormalization

from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Dropout
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import SeparableConv2D
from sklearn.utils import shuffle

In [2]:
# Parameter
image_size = (224, 224)  # Resize all images to this size


main_dir = os.path.expanduser("~/timeseries_data")  # Path to your main directory with class subfolders


def load_data(main_dir, image_size):
    data = []
    labels = []
    classes = sorted([cls for cls in os.listdir(main_dir) if os.path.isdir(os.path.join(main_dir, cls))])  # Filter directories only
    class_to_idx = {cls: idx for idx, cls in enumerate(classes)}  # Mapping class names to indices

    for cls in classes:
        class_dir = os.path.join(main_dir, cls)
        for img_file in os.listdir(class_dir):
            img_path = os.path.join(class_dir, img_file)
            if img_file.endswith(('.png', '.jpg', '.jpeg')):  # Ensure it's an image file
                img = Image.open(img_path).convert('RGB')  # Convert to RGB
                img = img.resize(image_size)  # Resize image
                data.append(np.array(img))
                labels.append(class_to_idx[cls])

    return np.array(data), np.array(labels), classes




In [3]:
# Generate a mapping for classes
classes = sorted([cls for cls in os.listdir(main_dir) if os.path.isdir(os.path.join(main_dir, cls))])
class_to_idx = {cls: idx for idx, cls in enumerate(classes)}  # {'AR_1_with_trend': 0, 'MA_1_without_trend': 1, ...}
print(f"Class to index mapping: {class_to_idx}")


Class to index mapping: {'ARMA_1_with_trend': 0, 'ARMA_1_without_trend': 1, 'ARMA_2_with_trend': 2, 'ARMA_2_without_trend': 3, 'ARMA_3_with_trend': 4, 'ARMA_3_without_trend': 5, 'AR_1_with_trend': 6, 'AR_1_without_trend': 7, 'AR_2_with_trend': 8, 'AR_2_without_trend': 9, 'AR_3_with_trend': 10, 'AR_3_without_trend': 11, 'MA_1_with_trend': 12, 'MA_1_without_trend': 13, 'MA_2_with_trend': 14, 'MA_2_without_trend': 15, 'MA_3_with_trend': 16, 'MA_3_without_trend': 17}


In [4]:
# Load the dataset
print("Loading dataset...")
data, labels, classes = load_data(main_dir, image_size)

# Normalize data
data = data / 255.0  # Normalize pixel values to [0, 1]

# Print class information
print(f"Classes: {classes}")
print(f"Number of classes: {len(classes)}")


Loading dataset...
Classes: ['ARMA_1_with_trend', 'ARMA_1_without_trend', 'ARMA_2_with_trend', 'ARMA_2_without_trend', 'ARMA_3_with_trend', 'ARMA_3_without_trend', 'AR_1_with_trend', 'AR_1_without_trend', 'AR_2_with_trend', 'AR_2_without_trend', 'AR_3_with_trend', 'AR_3_without_trend', 'MA_1_with_trend', 'MA_1_without_trend', 'MA_2_with_trend', 'MA_2_without_trend', 'MA_3_with_trend', 'MA_3_without_trend']
Number of classes: 18


#### Creating a subset of 5 classes

In [5]:
from sklearn.model_selection import train_test_split
import numpy as np

# Subset of classes to start with (e.g., 8 representative classes)
subset_classes = ['AR_1_with_trend', 'AR_2_without_trend', 
                  'MA_2_with_trend', 'MA_3_without_trend',
                  'ARMA_1_with_trend']

# Map subset classes to indices
subset_indices = [class_to_idx[cls] for cls in subset_classes]

# Filter data and labels for the subset
subset_mask = np.isin(labels, subset_indices)
data_subset = data[subset_mask]
labels_subset = labels[subset_mask]

# Reindex labels for the subset
labels_subset = np.array([subset_indices.index(lbl) for lbl in labels_subset])


data_subset = data_subset.astype('float32')




In [6]:
print(f"Data subset shape: {data_subset.shape}, size: {data_subset.nbytes / (1024**2):.2f} MB")
print(f"Labels subset shape: {labels_subset.shape}")


Data subset shape: (20000, 224, 224, 3), size: 11484.38 MB
Labels subset shape: (20000,)


#### Split into train, validation and test data

In [6]:
from sklearn.model_selection import train_test_split
import numpy as np
from tensorflow.keras.utils import Sequence


# Split indices instead of the whole dataset
train_idx, temp_idx = train_test_split(
    np.arange(len(labels_subset)), test_size=0.3, stratify=labels_subset, random_state=42
)

val_idx, test_idx = train_test_split(
    temp_idx, test_size=1/3, stratify=labels_subset[temp_idx], random_state=42
)

# Custom DataGenerator that uses indices
class IndexedDataGenerator(Sequence):
    def __init__(self, data, labels, indices, batch_size):
        self.data = data
        self.labels = labels
        self.indices = indices
        self.batch_size = batch_size

    def __len__(self):
        return int(np.ceil(len(self.indices) / self.batch_size))

    def __getitem__(self, idx):
        batch_indices = self.indices[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_data = self.data[batch_indices]
        batch_labels = self.labels[batch_indices]
        return batch_data, batch_labels

# Use indices to create separate generators
train_generator = IndexedDataGenerator(data_subset, labels_subset, train_idx, batch_size=32)
val_generator = IndexedDataGenerator(data_subset, labels_subset, val_idx, batch_size=32)
test_generator = IndexedDataGenerator(data_subset, labels_subset, test_idx, batch_size=32)


# Transfer learning and fine-tuning with ResNet50

### This script fine-tunes a ResNet50-based deep learning model for time series image classification. It first trains the model with frozen ResNet50 layers, adding custom classification layers on top. Transfer learning is applied using a low learning rate, and callbacks like early stopping, learning rate reduction, and model checkpointing are used. After initial training, the last 50 layers of ResNet50 are unfrozen for fine-tuning with an even lower learning rate to enhance performance while preventing overfitting.

In [47]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D, BatchNormalization
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint

# Parameters
IMAGE_SIZE = (224, 224, 3)  # Input size for ResNet50
NUM_CLASSES = len(subset_classes)  # Number of classes in the subset
BATCH_SIZE = 32
EPOCHS = 20

# Load the ResNet50 model with pretrained weights
base_model = ResNet50(
    input_shape=IMAGE_SIZE,
    include_top=False,  # Exclude the final classification layers
    weights="imagenet"
)

# Freeze the base model layers
for layer in base_model.layers:
    layer.trainable = False

# Add custom classification layers
x = base_model.output
x = GlobalAveragePooling2D()(x)  # Pool the feature maps to a single vector
x = BatchNormalization()(x)  # Normalize for stability
x = Dense(512, activation="relu")(x)  # Fully connected layer
x = Dropout(0.5)(x)  # Dropout for regularization
x = Dense(256, activation="relu")(x)
x = Dropout(0.5)(x)
output = Dense(NUM_CLASSES, activation="softmax")(x)  # Final classification layer

# Create the model
model = Model(inputs=base_model.input, outputs=output)

# Compile the model
model.compile(
    optimizer=Adam(learning_rate=1e-4),  # Use a lower learning rate for transfer learning
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

# Define callbacks
early_stopping = EarlyStopping(
    monitor="val_loss",
    patience=5,
    restore_best_weights=True,
    verbose=1
)

reduce_lr = ReduceLROnPlateau(
    monitor="val_loss",
    factor=0.5,
    patience=3,
    min_lr=1e-6,
    verbose=1
)

checkpoint = ModelCheckpoint(
    "subset_model.keras",
    monitor="val_accuracy",
    save_best_only=True,
    verbose=1
)

# Train the model on the subset
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=EPOCHS,
    callbacks=[early_stopping, reduce_lr, checkpoint],
    verbose=1
)


Epoch 1/20
438/438 ━━━━━━━━━━━━━━━━━━━━ 0s 735ms/step - accuracy: 0.3980 - loss: 1.1213
Epoch 1: val_accuracy improved from -inf to 0.44025, saving model to subset_model.keras
438/438 ━━━━━━━━━━━━━━━━━━━━ 415s 941ms/step - accuracy: 0.3980 - loss: 1.1212 - val_accuracy: 0.4403 - val_loss: 0.9981 - learning_rate: 1.0000e-04
Epoch 2/20
438/438 ━━━━━━━━━━━━━━━━━━━━ 0s 709ms/step - accuracy: 0.4296 - loss: 0.9796
Epoch 2: val_accuracy improved from 0.44025 to 0.46600, saving model to subset_model.keras
438/438 ━━━━━━━━━━━━━━━━━━━━ 401s 915ms/step - accuracy: 0.4296 - loss: 0.9796 - val_accuracy: 0.4660 - val_loss: 0.9305 - learning_rate: 1.0000e-04
Epoch 3/20
438/438 ━━━━━━━━━━━━━━━━━━━━ 0s 734ms/step - accuracy: 0.4342 - loss: 0.9566
Epoch 3: val_accuracy improved from 0.46600 to 0.48050, saving model to subset_model.keras
438/438 ━━━━━━━━━━━━━━━━━━━━ 416s 951ms/step - accuracy: 0.4342 - loss: 0.9566 - val_accuracy: 0.4805 - val_loss: 0.9207 - learning_rate: 1.0000e-04
Epoch 4/20
438/438 

In [51]:
# Unfreeze some layers in the base model for fine-tuning
for layer in base_model.layers[-50:]:  # Unfreeze the last 50 layers
    layer.trainable = True

# Recompile the model with a lower learning rate
model.compile(
    optimizer=Adam(learning_rate=1e-5),  # Lower learning rate for fine-tuning
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

# Fine-tune the model
history_finetune = model.fit(
    train_generator,  # Use the same training data
    validation_data=val_generator,
    epochs=15,  # Fine-tune for fewer epochs to avoid overfitting
    callbacks=[early_stopping, reduce_lr, checkpoint],
    verbose=1
)


Epoch 1/15
438/438 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.4176 - loss: 1.0157
Epoch 1: val_accuracy did not improve from 0.50525
438/438 ━━━━━━━━━━━━━━━━━━━━ 866s 2s/step - accuracy: 0.4176 - loss: 1.0157 - val_accuracy: 0.4375 - val_loss: 0.9349 - learning_rate: 1.0000e-05
Epoch 2/15
438/438 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.4503 - loss: 0.9345
Epoch 2: val_accuracy did not improve from 0.50525
438/438 ━━━━━━━━━━━━━━━━━━━━ 862s 2s/step - accuracy: 0.4503 - loss: 0.9345 - val_accuracy: 0.4415 - val_loss: 0.9331 - learning_rate: 1.0000e-05
Epoch 3/15
438/438 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.4747 - loss: 0.9176
Epoch 3: val_accuracy did not improve from 0.50525
438/438 ━━━━━━━━━━━━━━━━━━━━ 861s 2s/step - accuracy: 0.4746 - loss: 0.9176 - val_accuracy: 0.4600 - val_loss: 0.9086 - learning_rate: 1.0000e-05
Epoch 4/15
438/438 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.4951 - loss: 0.8946
Epoch 4: val_accuracy did not improve from 0.50525
438/438 ━━━━━━━━━━

# Incremental learning: dataset and model adaptation

### This script defines functions for dynamically updating a dataset and adapting a deep learning model for incremental learning. It includes a custom data generator to efficiently load indexed batches and a function to update the dataset by selecting remembered and new classes while maintaining class balance. Additionally, a function is provided to update the model by freezing layers and adjusting the output layer for a new number of classes, ensuring a smooth adaptation to changing datasets.

In [8]:
# Data generator 
class IndexedDataGenerator(Sequence):
    def __init__(self, data, labels, indices, batch_size):
        self.data = data
        self.labels = labels
        self.indices = indices
        self.batch_size = batch_size

    def __len__(self):
        return int(np.ceil(len(self.indices) / self.batch_size))

    def __getitem__(self, idx):
        batch_indices = self.indices[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_data = self.data[batch_indices]
        batch_labels = self.labels[batch_indices]
        return batch_data, batch_labels

In [9]:
# Function to update the dataset 
def update_dataset(data, labels, class_to_idx, remembered_classes, new_classes, batch_size=32):
    subset_classes = remembered_classes + new_classes
    subset_indices = [class_to_idx[cls] for cls in subset_classes]
    subset_mask = np.isin(labels, subset_indices)
    data_subset = data[subset_mask]
    labels_subset = labels[subset_mask]
    labels_subset = np.array([subset_indices.index(lbl) for lbl in labels_subset])

    # Split data into train, validation, and test indices
    train_idx, temp_idx = train_test_split(
        np.arange(len(labels_subset)), test_size=0.3, stratify=labels_subset, random_state=42
    )
    val_idx, test_idx = train_test_split(
        temp_idx, test_size=1/3, stratify=labels_subset[temp_idx], random_state=42
    )

    # Create new generators
    train_generator = IndexedDataGenerator(data_subset, labels_subset, train_idx, batch_size=batch_size)
    val_generator = IndexedDataGenerator(data_subset, labels_subset, val_idx, batch_size=batch_size)
    #test_generator = IndexedDataGenerator(data_subset, labels_subset, test_idx, batch_size=batch_size)

    return train_generator, val_generator, data_subset, labels_subset

In [10]:
# Function to update the model (freeze layers) 
def update_model_freeze(model, num_classes, learning_rate=1e-5):
    from tensorflow.keras.models import Model
    from tensorflow.keras.layers import Dense
    from tensorflow.keras.optimizers import Adam

    for layer in model.layers:
        layer.trainable = False  # Freeze all layers

    # Modify the output layer for the new number of classes
    new_output = Dense(num_classes, activation="softmax")(model.layers[-3].output)
    updated_model = Model(inputs=model.input, outputs=new_output)

    # Recompile the model with a smaller learning rate
    updated_model.compile(
        optimizer=Adam(learning_rate=learning_rate),
        loss="sparse_categorical_crossentropy",
        metrics=["accuracy"]
    )

    return updated_model

# Expanding model with new classes

### This script implements incremental learning across five increments by updating a pre-trained model with new classes while retaining knowledge from previous ones. It first loads a model trained on an initial subset of classes, then updates the dataset by selecting remembered and new classes at each step. The model is adjusted to accommodate the expanded class set while freezing earlier layers. Training is performed with callbacks for early stopping, learning rate reduction, and checkpointing to ensure stable learning and optimal performance throughout all five increments.

#### Increment 1

In [11]:
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping

# === Increment 1 Training ===

# Load the pre-trained subset model
model = load_model("subset_model.keras")  # Model trained on the initial 5 classes

# Define the current classes
current_classes = [
    'AR_1_with_trend', 'AR_2_without_trend',
    'MA_2_with_trend', 'MA_3_without_trend',
    'ARMA_1_with_trend'
]

# Define the first increment
increment_1 = ['AR_1_without_trend', 'MA_1_with_trend', 'MA_1_without_trend']


In [12]:
# === Increment 1 Training ===

# Update dataset
train_generator, val_generator, _, _ = update_dataset(
    data=data,  # Your dataset
    labels=labels,  # Your dataset labels
    class_to_idx=class_to_idx,  # Mapping from classes to indices
    remembered_classes=current_classes[-3:],  # Remember only the last 3 classes
    new_classes=increment_1,  # Add Increment 1 classes
    batch_size=16
    
)

# Update the model for Increment 1
model = update_model_freeze(
    model=model,
    num_classes=len(current_classes) + len(increment_1),  # 3 remembered + 3 new classes = 6 total
    learning_rate=1e-5
)

# Define callbacks for saving and early stopping
early_stopping = EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True, verbose=1)
reduce_lr = ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=3, min_lr=1e-6, verbose=1)
checkpoint = ModelCheckpoint(
    "final_8.keras",  # Save the updated model as "final_8.keras"
    monitor="val_accuracy",
    save_best_only=True,
    verbose=1
)

# Train the model
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=10,
    callbacks=[early_stopping, reduce_lr, checkpoint],
    verbose=1
)

# Update `current_classes` to include the new ones
current_classes = current_classes + increment_1


/Users/rebeccaganjineh/myenv_2/lib/python3.11/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
1050/1050 ━━━━━━━━━━━━━━━━━━━━ 0s 361ms/step - accuracy: 0.1804 - loss: 3.3579
Epoch 1: val_accuracy improved from -inf to 0.27437, saving model to final_8.keras
1050/1050 ━━━━━━━━━━━━━━━━━━━━ 490s 464ms/step - accuracy: 0.1804 - loss: 3.3574 - val_accuracy: 0.2744 - val_loss: 1.9357 - learning_rate: 1.0000e-05
Epoch 2/10
1050/1050 ━━━━━━━━━━━━━━━━━━━━ 0s 352ms/step - accuracy: 0.3006 - loss: 1.8195
Epoch 2: val_accuracy improved from 0.27437 to 0.36021, saving model to final_8.keras
1050/1050 ━━━━━━━━━━━━━━━━━━━━ 477s 454ms/step - accuracy: 0.3006 - loss: 1.8193 - val_accuracy: 0.3602 - val_loss: 1.3175 - learning_rate: 1.0000e-05
Epoch 3/10
1050/1050 ━━━━━━━━━━━━━━━━━━━━ 0s 357ms/step - accuracy: 0.3423 - loss: 1.4226
Epoch 3: val_accuracy improved from 0.36021 to 0.37708, saving model to final_8.keras
1050/1050 ━━━━━━━━━━━━━━━━━━━━ 481s 458ms/step - accuracy: 0.3423 - loss: 1.4226 - val_accuracy: 0.3771 - val_loss: 1.1919 - learning_rate: 1.0000e-05
Epoch 4/10
1050/1050 ━

#### Increment 2

In [11]:
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping

# === Increment 2 Training ===

# Load the pre-trained model
model = load_model("final_8.keras")  # Model trained on the initial 5 classes + increment 1

# Define the current classes
current_classes = [
    'AR_1_with_trend', 'AR_2_without_trend',
    'MA_2_with_trend', 'MA_3_without_trend',
    'ARMA_1_with_trend', 'AR_1_without_trend', 'MA_1_with_trend',
    'MA_1_without_trend'
]

# Define the second increment
increment_2 = ['AR_2_with_trend', 'MA_2_without_trend', 'ARMA_1_without_trend']


In [13]:
# === Increment 2 Training ===

# Update dataset
train_generator, val_generator, _, _ = update_dataset(
    data=data,  # Your dataset
    labels=labels,  # Your dataset labels
    class_to_idx=class_to_idx,  # Mapping from classes to indices
    remembered_classes=current_classes[-3:],  # Remember only the last 3 classes
    new_classes=increment_2,  # Add Increment 2 classes
    batch_size=16
)

# Update the model for Increment 2
model = update_model_freeze(
    model=model,
    num_classes=len(current_classes) + len(increment_2),  
    learning_rate=1e-5
)

# Define callbacks for saving and early stopping
early_stopping = EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True, verbose=1)
reduce_lr = ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=3, min_lr=1e-6, verbose=1)
checkpoint = ModelCheckpoint(
    "final_11.keras",  # Save the updated model as "final_11.keras"
    monitor="val_accuracy",
    save_best_only=True,
    verbose=1
)

# Train the model
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=10,
    callbacks=[early_stopping, reduce_lr, checkpoint],
    verbose=1
)

# Update `current_classes` to include the new ones
current_classes = current_classes + increment_2


Epoch 1/10


/Users/rebeccaganjineh/myenv_2/lib/python3.11/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


1050/1050 ━━━━━━━━━━━━━━━━━━━━ 0s 355ms/step - accuracy: 0.1514 - loss: 3.1123
Epoch 1: val_accuracy improved from -inf to 0.25896, saving model to final_11.keras
1050/1050 ━━━━━━━━━━━━━━━━━━━━ 484s 459ms/step - accuracy: 0.1514 - loss: 3.1120 - val_accuracy: 0.2590 - val_loss: 1.7729 - learning_rate: 1.0000e-05
Epoch 2/10
1050/1050 ━━━━━━━━━━━━━━━━━━━━ 0s 355ms/step - accuracy: 0.2526 - loss: 2.4462
Epoch 2: val_accuracy improved from 0.25896 to 0.36583, saving model to final_11.keras
1050/1050 ━━━━━━━━━━━━━━━━━━━━ 480s 457ms/step - accuracy: 0.2526 - loss: 2.4462 - val_accuracy: 0.3658 - val_loss: 1.5220 - learning_rate: 1.0000e-05
Epoch 3/10
1050/1050 ━━━━━━━━━━━━━━━━━━━━ 0s 359ms/step - accuracy: 0.3209 - loss: 2.2210
Epoch 3: val_accuracy improved from 0.36583 to 0.38458, saving model to final_11.keras
1050/1050 ━━━━━━━━━━━━━━━━━━━━ 484s 461ms/step - accuracy: 0.3209 - loss: 2.2210 - val_accuracy: 0.3846 - val_loss: 1.4020 - learning_rate: 1.0000e-05
Epoch 4/10
1050/1050 ━━━━━━━━━

#### Increment 3

In [11]:
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping

# === Increment 3 Training ===

# Load the pre-trained model
model = load_model("final_11.keras")  

# Define the current classes
current_classes = [
    'AR_1_with_trend', 'AR_2_without_trend',
    'MA_2_with_trend', 'MA_3_without_trend',
    'ARMA_1_with_trend', 'AR_1_without_trend', 'MA_1_with_trend',
    'MA_1_without_trend', 'AR_2_with_trend', 'MA_2_without_trend',
    'ARMA_1_without_trend'
]

# Define the second increment
increment_3 = ['AR_3_with_trend', 'AR_3_without_trend', 'MA_3_with_trend']


In [12]:
# === Increment 3 Training ===


# Update dataset
train_generator, val_generator, _, _ = update_dataset(
    data=data,  # Your dataset
    labels=labels,  # Your dataset labels
    class_to_idx=class_to_idx,  # Mapping from classes to indices
    remembered_classes=current_classes[-3:],  # Remember only the last 3 classes
    new_classes=increment_3,  # Add Increment 3 classes
    batch_size=16
)



# Update the model for Increment 3
model = update_model_freeze(
    model=model,
    num_classes=len(current_classes) + len(increment_3),  
    learning_rate=1e-5
)

# Define callbacks for saving and early stopping
early_stopping = EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True, verbose=1)
reduce_lr = ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=3, min_lr=1e-6, verbose=1)
checkpoint = ModelCheckpoint(
    "final_14.keras",  # Save the updated model as "final_14.keras"
    monitor="val_accuracy",
    save_best_only=True,
    verbose=1
)

# Train the model
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=20,
    callbacks=[early_stopping, reduce_lr, checkpoint],
    verbose=1
)

# Update `current_classes` to include the new ones
current_classes = current_classes + increment_3


/Users/rebeccaganjineh/myenv_2/lib/python3.11/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/20
1050/1050 ━━━━━━━━━━━━━━━━━━━━ 0s 360ms/step - accuracy: 0.1185 - loss: 2.7942
Epoch 1: val_accuracy improved from -inf to 0.33021, saving model to final_14.keras
1050/1050 ━━━━━━━━━━━━━━━━━━━━ 492s 466ms/step - accuracy: 0.1185 - loss: 2.7937 - val_accuracy: 0.3302 - val_loss: 1.6707 - learning_rate: 1.0000e-05
Epoch 2/20
1050/1050 ━━━━━━━━━━━━━━━━━━━━ 0s 383ms/step - accuracy: 0.3424 - loss: 1.5886
Epoch 2: val_accuracy improved from 0.33021 to 0.38708, saving model to final_14.keras
1050/1050 ━━━━━━━━━━━━━━━━━━━━ 516s 492ms/step - accuracy: 0.3424 - loss: 1.5885 - val_accuracy: 0.3871 - val_loss: 1.3754 - learning_rate: 1.0000e-05
Epoch 3/20
1050/1050 ━━━━━━━━━━━━━━━━━━━━ 0s 382ms/step - accuracy: 0.3777 - loss: 1.3550
Epoch 3: val_accuracy improved from 0.38708 to 0.39479, saving model to final_14.keras
1050/1050 ━━━━━━━━━━━━━━━━━━━━ 515s 491ms/step - accuracy: 0.3777 - loss: 1.3549 - val_accuracy: 0.3948 - val_loss: 1.2675 - learning_rate: 1.0000e-05
Epoch 4/20
1050/105

#### Increment 4

In [11]:
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping

# === Increment 4 Training ===

# Load the pre-trained model
model = load_model("final_14.keras")  

# Define the current classes
current_classes = [
    'AR_1_with_trend', 'AR_2_without_trend',
    'MA_2_with_trend', 'MA_3_without_trend',
    'ARMA_1_with_trend', 'AR_1_without_trend', 'MA_1_with_trend',
    'MA_1_without_trend', 'AR_2_with_trend', 'MA_2_without_trend',
    'ARMA_1_without_trend', 'AR_3_with_trend', 'AR_3_without_trend', 'MA_3_with_trend'
]

# Define the second increment
increment_4 = ['ARMA_2_with_trend', 'ARMA_2_without_trend', 'ARMA_3_with_trend',]


In [12]:
# === Increment 4 Training ===


# Update dataset
train_generator, val_generator, _, _ = update_dataset(
    data=data,  # Your dataset
    labels=labels,  # Your dataset labels
    class_to_idx=class_to_idx,  # Mapping from classes to indices
    remembered_classes=current_classes[-3:],  # Remember only the last 3 classes
    new_classes=increment_4,  # Add Increment 4 classes
    batch_size=16
)



# Update the model for Increment 4
model = update_model_freeze(
    model=model,
    num_classes=len(current_classes) + len(increment_4),  
    learning_rate=1e-5
)

# Define callbacks for saving and early stopping
early_stopping = EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True, verbose=1)
reduce_lr = ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=3, min_lr=1e-6, verbose=1)
checkpoint = ModelCheckpoint(
    "final_17.keras",  # Save the updated model as "final_17.keras"
    monitor="val_accuracy",
    save_best_only=True,
    verbose=1
)

# Train the model
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=20,
    callbacks=[early_stopping, reduce_lr, checkpoint],
    verbose=1
)

# Update `current_classes` to include the new ones
current_classes = current_classes + increment_4


Epoch 1/20


/Users/rebeccaganjineh/myenv_2/lib/python3.11/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


1050/1050 ━━━━━━━━━━━━━━━━━━━━ 0s 353ms/step - accuracy: 0.2936 - loss: 2.0851
Epoch 1: val_accuracy improved from -inf to 0.37625, saving model to final_17.keras
1050/1050 ━━━━━━━━━━━━━━━━━━━━ 482s 456ms/step - accuracy: 0.2936 - loss: 2.0846 - val_accuracy: 0.3762 - val_loss: 1.2878 - learning_rate: 1.0000e-05
Epoch 2/20
1050/1050 ━━━━━━━━━━━━━━━━━━━━ 0s 357ms/step - accuracy: 0.3721 - loss: 1.2682
Epoch 2: val_accuracy improved from 0.37625 to 0.38292, saving model to final_17.keras
1050/1050 ━━━━━━━━━━━━━━━━━━━━ 480s 457ms/step - accuracy: 0.3721 - loss: 1.2682 - val_accuracy: 0.3829 - val_loss: 1.2123 - learning_rate: 1.0000e-05
Epoch 3/20
1050/1050 ━━━━━━━━━━━━━━━━━━━━ 0s 356ms/step - accuracy: 0.3835 - loss: 1.2041
Epoch 3: val_accuracy improved from 0.38292 to 0.38604, saving model to final_17.keras
1050/1050 ━━━━━━━━━━━━━━━━━━━━ 481s 458ms/step - accuracy: 0.3835 - loss: 1.2041 - val_accuracy: 0.3860 - val_loss: 1.1844 - learning_rate: 1.0000e-05
Epoch 4/20
1050/1050 ━━━━━━━━━

#### Increment 5

In [13]:
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping

# === Increment 5 Training ===

# Load the pre-trained model
model = load_model("final_17.keras")  

# Define the current classes
current_classes = [
    'AR_1_with_trend', 'AR_2_without_trend',
    'MA_2_with_trend', 'MA_3_without_trend',
    'ARMA_1_with_trend', 'AR_1_without_trend', 'MA_1_with_trend',
    'MA_1_without_trend', 'AR_2_with_trend', 'MA_2_without_trend',
    'ARMA_1_without_trend', 'AR_3_with_trend', 'AR_3_without_trend', 'MA_3_with_trend',
    'ARMA_2_with_trend', 'ARMA_2_without_trend', 'ARMA_3_with_trend'
]

# Define the second increment
increment_5 = ['ARMA_3_without_trend' ]


In [14]:
# === Increment 5 Training ===


# Update dataset
train_generator, val_generator, _, _ = update_dataset(
    data=data,  # Your dataset
    labels=labels,  # Your dataset labels
    class_to_idx=class_to_idx,  # Mapping from classes to indices
    remembered_classes=current_classes[-3:],  # Remember only the last 3 classes
    new_classes=increment_5,  # Add Increment 5 classes
    batch_size=16
)



# Update the model for Increment 5
model = update_model_freeze(
    model=model,
    num_classes=len(current_classes) + len(increment_5),  
    learning_rate=1e-5
)

# Define callbacks for saving and early stopping
early_stopping = EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True, verbose=1)
reduce_lr = ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=3, min_lr=1e-6, verbose=1)
checkpoint = ModelCheckpoint(
    "final_18.keras",  # Save the updated model as "final_18.keras"
    monitor="val_accuracy",
    save_best_only=True,
    verbose=1
)

# Train the model
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=10,
    callbacks=[early_stopping, reduce_lr, checkpoint],
    verbose=1
)

# Update `current_classes` to include the new ones
current_classes = current_classes + increment_5


Epoch 1/10


/Users/rebeccaganjineh/myenv_2/lib/python3.11/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


700/700 ━━━━━━━━━━━━━━━━━━━━ 0s 348ms/step - accuracy: 0.3069 - loss: 2.1210
Epoch 1: val_accuracy improved from -inf to 0.51812, saving model to final_18.keras
700/700 ━━━━━━━━━━━━━━━━━━━━ 318s 450ms/step - accuracy: 0.3071 - loss: 2.1201 - val_accuracy: 0.5181 - val_loss: 0.9034 - learning_rate: 1.0000e-05
Epoch 2/10
700/700 ━━━━━━━━━━━━━━━━━━━━ 0s 351ms/step - accuracy: 0.5107 - loss: 0.8604
Epoch 2: val_accuracy improved from 0.51812 to 0.53375, saving model to final_18.keras
700/700 ━━━━━━━━━━━━━━━━━━━━ 316s 451ms/step - accuracy: 0.5107 - loss: 0.8604 - val_accuracy: 0.5337 - val_loss: 0.7766 - learning_rate: 1.0000e-05
Epoch 3/10
700/700 ━━━━━━━━━━━━━━━━━━━━ 0s 349ms/step - accuracy: 0.5223 - loss: 0.7704
Epoch 3: val_accuracy improved from 0.53375 to 0.54969, saving model to final_18.keras
700/700 ━━━━━━━━━━━━━━━━━━━━ 314s 449ms/step - accuracy: 0.5223 - loss: 0.7704 - val_accuracy: 0.5497 - val_loss: 0.7404 - learning_rate: 1.0000e-05
Epoch 4/10
700/700 ━━━━━━━━━━━━━━━━━━━━ 0s